# Predicting March Madness Winners

In [1]:
import numpy as np
import pandas as pd

import random
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from scipy.stats import norm
import xgboost as xgb
import graphviz

from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile
from datetime import datetime
import pytz
import itertools
import os

In [12]:
api = KaggleApi()
api.authenticate()

In [13]:
def write_to_csv(df, outname, outdir, index=None):
    '''
    inputs:
    output:
    '''
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)    
    df.to_csv(fullname, index=index) 
    return

def save_model_to_dir(model, outname, outdir):
    '''
    inputs:
    output:
    '''
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)    
    model.save_model(fullname)
    return

## <u>Data Processing</u>

### Pull Data from Kaggle
Use the Kaggle API to list and download relevant files for 2022 March Madness Tournament (men's or women's).

In [76]:
bracket_type = 'womens'
bi = bracket_type[0].upper()

In [77]:
# api.competitions_list(search='march')
api.competition_list_files('march-machine-learning-mania-2023')

[MSecondaryTourneyTeams.csv,
 Conferences.csv,
 MSeasons.csv,
 Cities.csv,
 MTeamSpellings.csv,
 MRegularSeasonCompactResults.csv,
 MNCAATourneySlots.csv,
 MNCAATourneyDetailedResults.csv,
 SampleSubmission2023.csv,
 MTeamConferences.csv,
 MTeamCoaches.csv,
 MNCAATourneySeeds.csv,
 MNCAATourneyCompactResults.csv,
 MSecondaryTourneyCompactResults.csv,
 MTeams.csv,
 MGameCities.csv,
 MMasseyOrdinals_thru_Season2023_Day128.csv,
 MRegularSeasonDetailedResults.csv,
 MConferenceTourneyGames.csv,
 MNCAATourneySeedRoundSlots.csv]

<b>Note:</b> Only run this section once to download files

In [19]:
# api.competition_download_files('march-machine-learning-mania-2023')

# zf = ZipFile('march-machine-learning-mania-2023.zip')
# zf.extractall('Data/') #save files in selected folder
# zf.close()

### Extract Regular Season Matchup Data

In [78]:
season_data = pd.read_csv('Data/{}RegularSeasonDetailedResults.csv'.format(bi, bi))
season_data.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

### Functions to Transform Regular Season Matchup Data

In [22]:
def get_continuous_data(data, min_season=2010, max_season=2023, last_n=None, vocal=True):
    '''
    inputs: data (dataframe), matchup data with each teams' basic stats as features
            min_season (int), minimum season to consider in data (default 2010)
            max_season (int), maximum season to consider in data (default 2022)
            last_n (int), number of days to consider in look-back window (default None -- all days considered)
            vocal (bool), prints updates if True (default True)
    output: rolling averages of "flattened" version of data (each team in matchup given its own row) 
            with advanced stats added (Win%, Away-Win%, Away/Neutral-Win%, Min, G, FG%, FT%, 3P%, 3Pr, FTr, 
            eFG%, TS%, Possession, Pace, Reb%, O-Reb%, Ast%, Stl%, Blk%, TO%, Off/Def Rating for Team/Opponent)
    '''
    if vocal:
        print('---- PROCESS CONTINOUS SEASON DATA ----\nSTART:', datetime.now(pytz.timezone('US/Pacific')))
        
    cols1 = ['Season','DayNum','WTeamID','WLoc','NumOT','WScore','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA',\
             'WOR','WDR','WAst','WTO','WStl','WBlk','WPF','LScore','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA',\
             'LOR','LDR','LAst','LTO','LStl','LBlk','LPF']
    cols2 = ['Season','DayNum','LTeamID','WLoc','NumOT','LScore','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA',\
             'LOR','LDR','LAst','LTO','LStl','LBlk','LPF','WScore','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA',\
             'WOR','WDR','WAst','WTO','WStl','WBlk','WPF']

    cols = ['Season','DayNum','TeamID','Loc','NumOT','TScore','TFGM','TFGA','TFGM3','TFGA3','TFTM','TFTA',
            'TOR','TDR','TAst','TTO','TStl','TBlk','TPF','OScore','OFGM','OFGA','OFGM3','OFGA3','OFTM','OFTA',\
            'OOR','ODR','OAst','OTO','OStl','OBlk','OPF']

    all_cols = ['Season','TeamID','NumOT','TScore','TFGM','TFGA','TFGM3','TFGA3','TFTM','TFTA','TOR','TDR',\
                'TAst','TTO','TStl','TBlk','TPF','OScore','OFGM','OFGA','OFGM3','OFGA3','OFTM','OFTA','OOR','ODR',\
                'OAst','OTO','OStl','OBlk','OPF','Wins','G','WP','AWP','ANWP','Min','TFGP','TFGP3','TFTP',\
                'TFG3R','TFTR','TEFG','TTFG','OFGP','OFGP3','OFTP','OFG3R','OFTR','OEFG','OTFG','TPoss','TPace',\
                'OPoss','OPace','TTRP','TORP','TAstP','TStlP','TBlkP','TTOP','OTRP','OORP','OAstP','OStlP',\
                'OBlkP','OTOP','OffRtg','DefRtg','DayNum']

    dfc = pd.DataFrame(columns=all_cols)
    
    # Iterate overe each season of data
    for seas in range(min_season, max_season+1):
        max_daynum = max(data[(data['Season']==seas)]['DayNum'])
        for daynum in range(1,max_daynum+1):
            days_back = last_n
            if days_back is None:
                days_back = daynum

            df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]

            if len(df) > 0:
                tm1 = df[cols1].rename(columns=dict(zip(cols1, cols)))
                tm2 = df[cols2].rename(columns=dict(zip(cols2, cols)))

                # Calculate total wins
                tm1['Wins'] = 1
                tm2['Wins'] = 0

                # Calculate total away wins and losses (Loc will be H for the losing team)
                tm1['AWins'] = tm1.apply(lambda row: 1 if row['Loc'] == 'A' else 0, axis=1)
                tm2['AWins'] = 0
                tm1['ALosses'] = 0
                tm2['ALosses'] = tm2.apply(lambda row: 1 if row['Loc'] == 'H' else 0, axis=1)

                # Calculate total neutral wins and losses
                tm1['NWins'] = tm1.apply(lambda row: 1 if row['Loc'] == 'N' else 0, axis=1)
                tm2['NWins'] = 0
                tm1['NLosses'] = 0
                tm2['NLosses'] = tm2.apply(lambda row: 1 if row['Loc'] == 'N' else 0, axis=1)

                tm = tm1.append(tm2)

                tm['G'] = 1

                agg_funcs = 29*['mean']+6*['sum']
                tm = tm.groupby(['Season','TeamID'], as_index=False).agg(dict(zip(cols[4:]+['G','Wins',\
                                                        'AWins','ALosses','NWins','NLosses'], agg_funcs)))

                # Game statistics
                tm['WP'] = tm['Wins']/tm['G'] # Win%
                tm['AWP'] = tm['AWins']/(tm['AWins']+tm['ALosses']) # Away Win%
                tm['ANWP'] = (tm['AWins']+tm['NWins'])/(tm['AWins']+tm['ALosses']+tm['NWins']+tm['NLosses']) # Away/Neutral Win%
                tm['Min'] = 40*tm['G']+5*tm['NumOT'] # Min

                # Team shooting percentages
                tm['TFGP'] = tm['TFGM']/tm['TFGA'] # Team FG%
                tm['TFGP3'] = tm['TFGM3']/tm['TFGA3'] # Team 3P%
                tm['TFTP'] = tm['TFTM']/tm['TFTA'] # Team FT%
                tm['TFG3R'] = tm['TFGA3']/tm['TFGA'] # Team 3P Rate
                tm['TFTR'] = tm['TFTA']/tm['TFGA'] # Team FT Rate
                tm['TEFG'] = (0.5*tm['TFGM3']+tm['TFGM'])/tm['TFGA'] # Team Effective FG%
                tm['TTFG'] = tm['TScore']/(2*(0.44*tm['TFTA']+tm['TFGA'])) # Team TS%

                # Opponent shooting percentages
                tm['OFGP'] = tm['OFGM']/tm['OFGA'] # Opponent FG%
                tm['OFGP3'] = tm['OFGM3']/tm['OFGA3'] # Opponent 3P%
                tm['OFTP'] = tm['OFTM']/tm['OFTA'] # Opponent FT%
                tm['OFG3R'] = tm['OFGA3']/tm['OFGA'] # Opponent 3P Rate
                tm['OFTR'] = tm['OFTA']/tm['OFGA'] # Opponent FT Rate
                tm['OEFG'] = (0.5*tm['OFGM3']+tm['OFGM'])/tm['OFGA'] # Opponent Effective FG%
                tm['OTFG'] = tm['OScore']/(2*(0.44*tm['OFTA']+tm['OFGA'])) # Opponent TS%

                # Team possession stats
                tm['TPoss'] = tm['TFGA']-tm['TOR']+tm['TTO']+0.4*tm['TFTA'] # Team Possession
                tm['TPace'] = 40*tm['TPoss']/tm['Min'] # Team Pace

                # Opponent possession stats
                tm['OPoss'] = tm['OFGA']-tm['OOR']+tm['OTO']+0.4*tm['OFTA'] # Opponent Possession
                tm['OPace'] = 40*tm['OPoss']/tm['Min'] # Opponent Pace

                # Team stat percentages
                tm['TTRP'] = (tm['TOR']+tm['TDR'])/(tm['TOR']+tm['TDR']+tm['OOR']+tm['ODR']) # Team Rebound%
                tm['TORP'] = tm['TOR']/(tm['TOR']+tm['ODR']) # Team Offensive-Rebound%
                tm['TAstP'] = tm['TAst']/tm['TFGM'] # Team Ast%
                tm['TStlP'] = tm['TStl']/tm['OPoss'] # Team Stl%
                tm['TBlkP'] = tm['TBlk']/(tm['OFGA']-tm['OFGA3']) # Team Blk%
                tm['TTOP'] = tm['TTO']/tm['TPoss'] # Team TO%

                # Opponent stat percentages
                tm['OTRP'] = (tm['OOR']+tm['ODR'])/(tm['TOR']+tm['TDR']+tm['OOR']+tm['ODR']) # Opponent Rebound%
                tm['OORP'] = tm['OOR']/(tm['OOR']+tm['TDR']) # Opponent Offensive-Rebound%
                tm['OAstP'] = tm['OAst']/tm['OFGM'] # Opponent Ast%
                tm['OStlP'] = tm['OStl']/tm['TPoss'] # Opponent Stl%
                tm['OBlkP'] = tm['OBlk']/(tm['TFGA']-tm['TFGA3']) # Opponent Blk%
                tm['OTOP'] = tm['OTO']/tm['OPoss'] # Opponent TO%

                # Ratings
                tm['OffRtg'] = tm['TScore']/tm['TPoss'] # Offensive Rating
                tm['DefRtg'] = tm['OScore']/tm['OPoss'] # Defensive Rating

                tm['DayNum'] = daynum

                tm = tm.drop(columns=['AWins','ALosses','NWins','NLosses'])

                dfc = dfc.append(tm)
                
        if vocal:
            print('{} season completed.'.format(seas))
    if vocal:
        print('FINISH:', datetime.now(pytz.timezone('US/Pacific')))
            
    return dfc

def get_matchup_dict(data, min_season=2010, max_season=2023, max_daynum=132, 
                     min_team=1101, max_team=1499, last_n=30, vocal=True):
    '''
    '''
    if vocal:
        print('---- COLLECT TEAM MATCHUP HISTORY ----\nSTART:', datetime.now(pytz.timezone('US/Pacific')))
    # Get matchup history of all teams on any given day
    matchup_d = {}
    for seas in range(min_season, max_season+1):
        d_day = {}
        for daynum in range(max_daynum+1):
            days_back = last_n
            if days_back is None:
                days_back = daynum
            df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
            d_team = {}
            for t in range(min_team, max_team+1):
                d_team[t] = list(df[df['WTeamID']==t]['LTeamID'].append(df[df['LTeamID']==t]['WTeamID']))
            d_day[daynum] = d_team
        matchup_d[seas] = d_day
        if vocal:
            print('{} season completed.'.format(seas))
    if vocal:
        print('FINISH:', datetime.now(pytz.timezone('US/Pacific')))
    return matchup_d

def get_opp_matchup_dict(matchup_d, min_season=2010, max_season=2023, max_daynum=132, 
                         min_team=1101, max_team=1499, vocal=True):
    '''
    '''
    if vocal:
        print('---- COLLECT OPPONENT MATCHUP HISTORY ----\nSTART:', datetime.now(pytz.timezone('US/Pacific')))
    # Get matchup history of all teams' opponents on any given day
    opp_matchup_d = {}
    for seas in range(min_season, max_season+1):
        d_day = {}
        for daynum in range(max_daynum+1):
            d_team = {}
            for t in range(min_team, max_team+1):
                opp_matchups = []
                for o in matchup_d[seas][daynum][t]:
                    opp_matchups += matchup_d[seas][daynum][o]
                d_team[t] = opp_matchups
            d_day[daynum] = d_team
        opp_matchup_d[seas] = d_day
        if vocal:
            print('{} season completed.'.format(seas))
    if vocal:
        print('FINISH:', datetime.now(pytz.timezone('US/Pacific')))
    return opp_matchup_d

def get_win_pct_dict(data_cont):
    '''
    '''
    return {h: {g: f.groupby('TeamID')['WP'].apply(float).to_dict() \
                for g, f in g.groupby('DayNum')} for h, g in data_cont.groupby('Season')}

def get_elo_dict(data, min_season=2010, max_season=2023, max_daynum=132, min_team=1101, max_team=1499, vocal=True):
    '''
    '''
    if vocal:
        print('---- CALCULATE SIMPLE ELO SCORE ----\nSTART:', datetime.now(pytz.timezone('US/Pacific')))
    # Get matchup history of all teams on any given day
    elo_d = {}
    for seas in range(min_season, max_season+1):
        d_day = {}
        for daynum in range(max_daynum+1):
            df = data[(data['Season']==seas)&(data['DayNum']<daynum)]
            d_team = {}
            for t in range(min_team, max_team+1):
                # Initialize Elo if first game of season
                if daynum == 0:
                    if seas == min_season:
                        d_team[t] = [1500,0]
                    else:
                        d_team[t] = [(2*elo_d[seas-1][max_daynum-1][t][0] + 1500) / 3, 0]
                # Calculate Elo based on previous Elo of both teams involved
                else:
                    # Create temporary datafame to store past opponents, day, and result (win or loss)
                    temp_df1 = df[df['WTeamID']==t][['LTeamID','DayNum']].rename(columns={'LTeamID':'TeamID'})
                    temp_df1['W'] = 1
                    temp_df2 = df[df['LTeamID']==t][['WTeamID','DayNum']].rename(columns={'WTeamID':'TeamID'})
                    temp_df2['W'] = 0
                    temp_df = temp_df1.append(temp_df2)
                    # Get number of games in season
                    num_games = len(temp_df)
                    # If no new game on this day, keep Elo the same
                    if d_day[daynum-1][t][1] == num_games:
                        d_team[t] = [d_day[daynum-1][t][0], num_games]
                    # Otherwise, calculate Elo based on previous Elo and previous opponent Elo
                    else:
                        # Locate the most recent opponent and result (win or loss)
                        last_opp = temp_df.loc[temp_df['DayNum'].idxmax()]['TeamID']
                        result = temp_df.loc[temp_df['DayNum'].idxmax()]['W']
                        score = d_day[daynum-1][t][0]*(num_games-1) + d_day[daynum-1][last_opp][0]
                        score = (score + (2*result-1) * 400) / num_games
                        d_team[t] = [score, num_games]
            d_day[daynum] = d_team
        elo_d[seas] = d_day
        if vocal:
            print('{} season completed.'.format(seas))
    if vocal:
        print('FINISH:', datetime.now(pytz.timezone('US/Pacific')))
    return {s: {d: {t: v[0] for t, v in dict1.items()} for d, dict1 in dict0.items()} for s, dict0 in elo_d.items()}

def get_carmelo_dict(data, min_season=2010, max_season=2023, max_daynum=132, min_team=1101, max_team=1499, vocal=True):
    '''
    '''
    if vocal:
        print('---- CALCULATE 538 ELO SCORE ----\nSTART:', datetime.now(pytz.timezone('US/Pacific')))
    # Get matchup history of all teams on any given day
    elo_d = {}
    for seas in range(min_season, max_season+1):
        d_day = {}
        for daynum in range(max_daynum+1):
            df = data[(data['Season']==seas)&(data['DayNum']<daynum)]
            d_team = {}
            for t in range(min_team, max_team+1):
                # Initialize Elo if first game of season
                if daynum == 0:
                    if seas == min_season:
                        d_team[t] = [1500,0]
                    else:
                        d_team[t] = [(2*elo_d[seas-1][max_daynum-1][t][0] + 1500) / 3, 0]
                # Calculate CarmElo based on previous CarmElo of both teams involved
                else:
                    # Create temporary datafame to store past opponents, day, scores, and result (win or loss)
                    temp_df1 = df[df['WTeamID']==t][['LTeamID','DayNum','WScore','LScore','WLoc']]
                    temp_df1 = temp_df1.rename(columns={'LTeamID':'TeamID','WScore':'Score1','LScore':'Score2'})
                    temp_df1['WLoc'] = temp_df1.apply(lambda x: 1 if x['WLoc'] == 'H' else 0, axis=1)
                    temp_df2 = df[df['LTeamID']==t][['WTeamID','DayNum','LScore','WScore','WLoc']]
                    temp_df2 = temp_df2.rename(columns={'WTeamID':'TeamID','LScore':'Score1','WScore':'Score2'})
                    temp_df2['WLoc'] = temp_df2.apply(lambda x: 1 if x['WLoc'] == 'A' else 0, axis=1)
                    temp_df = temp_df1.append(temp_df2)
                    temp_df['W'] = temp_df.apply(lambda x: 1 if len(temp_df) > 0 and x['Score1'] > x['Score2'] else 0, axis=1)
                    # Get number of games in season
                    num_games = len(temp_df)
                    # If no new game on this day, keep CarmElo the same
                    if d_day[daynum-1][t][1] == num_games:
                        d_team[t] = [d_day[daynum-1][t][0], num_games]
                    # Otherwise, calculate CarmElo based on previous Elo and previous opponent CarmElo
                    else:
                        # Locate the most recent opponent and result (win or loss)
                        last_opp = temp_df.loc[temp_df['DayNum'].idxmax()]['TeamID']
                        last_result = temp_df.loc[temp_df['DayNum'].idxmax()]['W']
                        last_score1 = temp_df.loc[temp_df['DayNum'].idxmax()]['Score1']
                        last_score2 = temp_df.loc[temp_df['DayNum'].idxmax()]['Score2']
                        was_home = temp_df.loc[temp_df['DayNum'].idxmax()]['WLoc']
                        # Calculate CarmElo in steps
                        team_elo = d_day[daynum-1][t][0]
                        opp_elo = d_day[daynum-1][last_opp][0]
                        # Add 100 to CarmElo if team was at home
                        elo_diff = (opp_elo + (1-was_home)*100 - team_elo + was_home*100)
                        e_team = 1 / (1 + 10**(elo_diff/400))
                        k = 20 * (abs(last_score1 - last_score2) + 3)**0.8 / (7.5 + 0.006*abs(elo_diff))
                        score = k*(last_result - e_team) + team_elo
                        d_team[t] = [score, num_games]
            d_day[daynum] = d_team
        elo_d[seas] = d_day
        if vocal:
            print('{} season completed.'.format(seas))
    if vocal:
        print('FINISH:', datetime.now(pytz.timezone('US/Pacific')))
    return {s: {d: {t: v[0] for t, v in dict1.items()} for d, dict1 in dict0.items()} for s, dict0 in elo_d.items()}

def add_sos_var(data, wp_d, matchup_d, opp_matchup_d, vocal=True):
    '''
    '''
    if vocal:
        print('---- ADD SOS FEATURES TO DATA ----\nSTART:', datetime.now(pytz.timezone('US/Pacific')))
    # Calculate opponnent win percentage
    data['OWP'] = data.apply(lambda row: np.mean([wp_d[row['Season']][row['DayNum']][x] \
                 for x in matchup_d[row['Season']][row['DayNum']][row['TeamID']]]), axis=1)
    # Calculate opponent's opponent win percentage
    data['OOWP'] = data.apply(lambda row: np.mean([wp_d[row['Season']][row['DayNum']][x] \
                 for x in opp_matchup_d[row['Season']][row['DayNum']][row['TeamID']]]), axis=1)
    # Calculate strength of schedule from OWP and OOWP
    data['SOS'] = (2*data['OWP']+data['OOWP'])/3
    if vocal:
        print('FINISH:', datetime.now(pytz.timezone('US/Pacific')))
    return data

def add_elo_var(data, elo_d, matchup_d, elo_var_name='Elo', vocal=True):
    '''
    '''
    if vocal:
        print('---- ADD ELO FEATURES TO DATA ----\nSTART:', datetime.now(pytz.timezone('US/Pacific')))
    data['T'+elo_var_name] = data.apply(lambda row: elo_d[row['Season']][row['DayNum']][row['TeamID']], axis=1)
    data['O'+elo_var_name] = data.apply(lambda row: np.mean([elo_d[row['Season']][row['DayNum']][x] \
                 for x in matchup_d[row['Season']][row['DayNum']][row['TeamID']]]), axis=1)
    if vocal:
        print('FINISH:', datetime.now(pytz.timezone('US/Pacific')))
    return data

### Create Continuous Season Dataset

In [79]:
MIN_TEAM = 1101
MAX_TEAM = 1499

if bracket_type == 'womens':
    MIN_TEAM += 2000
    MAX_TEAM += 2000

In [80]:
elo_dict = get_elo_dict(season_data, min_team=MIN_TEAM, max_team=MAX_TEAM)

---- CALCULATE SIMPLE ELO SCORE ----
START: 2023-03-14 11:14:36.577726-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 11:32:30.788165-07:00


In [81]:
carmelo_dict = get_carmelo_dict(season_data, min_team=MIN_TEAM, max_team=MAX_TEAM)

---- CALCULATE 538 ELO SCORE ----
START: 2023-03-14 11:32:31.076219-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 12:00:48.782814-07:00


In [82]:
continuous_data_all = get_continuous_data(season_data)
continuous_data_all.head()

---- PROCESS CONTINOUS SEASON DATA ----
START: 2023-03-14 12:00:49.011142-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 12:03:38.665100-07:00


,Season,TeamID,NumOT,TScore,TFGM,TFGA,TFGM3,TFGA3,TFTM,TFTA,...,TTOP,OTRP,OORP,OAstP,OStlP,OBlkP,OTOP,OffRtg,DefRtg,DayNum
0,2010,3102,0.0,46.0,15.0,47.0,5.0,17.0,11.0,20.0,...,0.367647,0.576923,0.500000,0.520000,0.147059,0.033333,0.263930,0.676471,0.953079,12
1,2010,3103,0.0,63.0,23.0,54.0,5.0,9.0,12.0,19.0,...,0.258621,0.513514,0.297297,0.550000,0.100575,0.133333,0.328571,0.905172,0.700000,12
2,2010,3104,0.0,73.0,26.0,62.0,5.0,12.0,16.0,28.0,...,0.259067,0.459770,0.311111,0.280000,0.051813,0.040000,0.250627,0.945596,0.852130,12
3,2010,3105,0.0,61.0,22.0,57.0,3.0,12.0,14.0,21.0,...,0.344828,0.448718,0.289474,0.633333,0.198939,0.133333,0.198939,0.809019,0.954907,12
4,2010,3107,0.0,56.0,22.0,57.0,5.0,17.0,7.0,9.0,...,0.326087,0.538462,0.321429,0.448276,0.190217,0.125000,0.261708,0.760870,1.046832,12


In [83]:
matchup_dict_all = get_matchup_dict(season_data, min_team=MIN_TEAM, max_team=MAX_TEAM)

---- COLLECT TEAM MATCHUP HISTORY ----
START: 2023-03-14 12:03:38.690771-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 12:08:28.876122-07:00


In [84]:
opp_matchup_dict_all = get_opp_matchup_dict(matchup_dict_all, min_team=MIN_TEAM, max_team=MAX_TEAM)

---- COLLECT OPPONENT MATCHUP HISTORY ----
START: 2023-03-14 12:08:29.039235-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 12:08:31.790361-07:00


In [85]:
# Add columns to dataframe
wp_dict_all = get_win_pct_dict(continuous_data_all)
continuous_data_all = add_sos_var(continuous_data_all, wp_dict_all, matchup_dict_all, opp_matchup_dict_all)
continuous_data_all = add_elo_var(continuous_data_all, elo_dict, matchup_dict_all, elo_var_name='Elo')
continuous_data_all = add_elo_var(continuous_data_all, carmelo_dict, matchup_dict_all, elo_var_name='CarmElo')

# Modify dataframe
continuous_data_all = continuous_data_all.apply(pd.to_numeric, errors='coerce', axis=1)
continuous_data_all = continuous_data_all.reset_index(drop=True).drop(columns=['Wins'])

# Write dataframe
write_to_csv(continuous_data_all, 'regular_season_continuous.csv', 'Data/{}Transformed/'.format(bi))
continuous_data_all.head()

---- ADD SOS FEATURES TO DATA ----
START: 2023-03-14 12:08:42.927654-07:00


/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


FINISH: 2023-03-14 12:10:55.319781-07:00
---- ADD ELO FEATURES TO DATA ----
START: 2023-03-14 12:10:55.321158-07:00


/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


FINISH: 2023-03-14 12:11:27.064320-07:00
---- ADD ELO FEATURES TO DATA ----
START: 2023-03-14 12:11:27.065639-07:00


/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


FINISH: 2023-03-14 12:11:58.144989-07:00


,Season,TeamID,NumOT,TScore,TFGM,TFGA,TFGM3,TFGA3,TFTM,TFTA,...,OffRtg,DefRtg,DayNum,OWP,OOWP,SOS,TElo,OElo,TCarmElo,OCarmElo
0,2010.0,3102.0,0.0,46.0,15.0,47.0,5.0,17.0,11.0,20.0,...,0.676471,0.953079,12.0,1.0,0.0,0.666667,1100.0,1900.0,1489.463237,1518.737308
1,2010.0,3103.0,0.0,63.0,23.0,54.0,5.0,9.0,12.0,19.0,...,0.905172,0.700000,12.0,0.0,1.0,0.333333,1900.0,1100.0,1515.245027,1491.427091
2,2010.0,3104.0,0.0,73.0,26.0,62.0,5.0,12.0,16.0,28.0,...,0.945596,0.852130,12.0,0.0,1.0,0.333333,1900.0,1100.0,1508.341440,1495.309263
3,2010.0,3105.0,0.0,61.0,22.0,57.0,3.0,12.0,14.0,21.0,...,0.809019,0.954907,12.0,1.0,0.0,0.666667,1100.0,1900.0,1492.660415,1513.051832
4,2010.0,3107.0,0.0,56.0,22.0,57.0,5.0,17.0,7.0,9.0,...,0.760870,1.046832,12.0,1.0,0.0,0.666667,1100.0,1900.0,1489.081793,1519.415623


In [86]:
season_end_data_all = continuous_data_all.loc[continuous_data_all.groupby(['Season','TeamID'])['DayNum'].idxmax()]
write_to_csv(season_end_data_all, 'regular_season_end.csv', 'Data/{}Transformed/'.format(bi))
season_end_data_all.head()

,Season,TeamID,NumOT,TScore,TFGM,TFGA,TFGM3,TFGA3,TFTM,TFTA,...,OffRtg,DefRtg,DayNum,OWP,OOWP,SOS,TElo,OElo,TCarmElo,OCarmElo
40846,2010.0,3102.0,0.000000,51.821429,19.142857,53.142857,4.571429,16.071429,8.964286,13.214286,...,0.813797,1.123031,132.0,0.612388,0.521812,0.582196,1162.693059,1655.076964,1369.435304,1663.984799
40847,2010.0,3103.0,0.033333,62.233333,22.233333,55.266667,3.933333,12.466667,13.833333,19.600000,...,0.926459,0.882585,132.0,0.543819,0.484570,0.524069,1514.272422,1518.182900,1649.115755,1630.713164
40848,2010.0,3104.0,0.103448,64.137931,24.724138,62.103448,4.344828,14.724138,10.344828,16.793103,...,0.872502,0.914009,132.0,0.650157,0.563932,0.621415,1568.523895,1793.482995,1565.205331,1710.970642
40849,2010.0,3105.0,0.000000,60.777778,20.370370,50.851852,3.037037,9.888889,17.000000,24.333333,...,0.844657,0.893204,132.0,0.415217,0.431164,0.420533,1352.886412,1297.387046,1518.972030,1521.725721
40850,2010.0,3106.0,0.034483,55.137931,18.448276,53.000000,2.689655,9.586207,15.551724,24.000000,...,0.814404,0.853321,132.0,0.492054,0.402033,0.462047,1324.703218,1339.977534,1528.690495,1551.283403


### Create Last-30-Day Continuous Dataset

In [87]:
continuous_data_l30 = get_continuous_data(season_data, last_n=30)
continuous_data_l30.head()

---- PROCESS CONTINOUS SEASON DATA ----
START: 2023-03-14 12:13:34.769418-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 12:15:45.657165-07:00


,Season,TeamID,NumOT,TScore,TFGM,TFGA,TFGM3,TFGA3,TFTM,TFTA,...,TTOP,OTRP,OORP,OAstP,OStlP,OBlkP,OTOP,OffRtg,DefRtg,DayNum
0,2010,3102,0.0,46.0,15.0,47.0,5.0,17.0,11.0,20.0,...,0.367647,0.576923,0.500000,0.520000,0.147059,0.033333,0.263930,0.676471,0.953079,12
1,2010,3103,0.0,63.0,23.0,54.0,5.0,9.0,12.0,19.0,...,0.258621,0.513514,0.297297,0.550000,0.100575,0.133333,0.328571,0.905172,0.700000,12
2,2010,3104,0.0,73.0,26.0,62.0,5.0,12.0,16.0,28.0,...,0.259067,0.459770,0.311111,0.280000,0.051813,0.040000,0.250627,0.945596,0.852130,12
3,2010,3105,0.0,61.0,22.0,57.0,3.0,12.0,14.0,21.0,...,0.344828,0.448718,0.289474,0.633333,0.198939,0.133333,0.198939,0.809019,0.954907,12
4,2010,3107,0.0,56.0,22.0,57.0,5.0,17.0,7.0,9.0,...,0.326087,0.538462,0.321429,0.448276,0.190217,0.125000,0.261708,0.760870,1.046832,12


In [88]:
matchup_dict_l30 = get_matchup_dict(season_data, last_n=30, min_team=MIN_TEAM, max_team=MAX_TEAM)

---- COLLECT TEAM MATCHUP HISTORY ----
START: 2023-03-14 12:15:45.691005-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 12:20:34.378524-07:00


In [89]:
opp_matchup_dict_l30 = get_opp_matchup_dict(matchup_dict_l30, min_team=MIN_TEAM, max_team=MAX_TEAM)

---- COLLECT OPPONENT MATCHUP HISTORY ----
START: 2023-03-14 12:20:34.604273-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 12:20:35.823327-07:00


In [90]:
# Add columns to dataframe
wp_dict_l30 = get_win_pct_dict(continuous_data_l30)
continuous_data_l30 = add_sos_var(continuous_data_l30, wp_dict_l30, matchup_dict_l30, opp_matchup_dict_l30)

# Modify dataframe
continuous_data_l30 = continuous_data_l30.apply(pd.to_numeric, errors='coerce', axis=1)
continuous_data_l30 = continuous_data_l30.reset_index(drop=True).drop(columns=['Wins'])

# Write dataframe
write_to_csv(continuous_data_l30, 'regular_season_continuous_last30.csv', 'Data/{}Transformed/'.format(bi))
continuous_data_l30.head()

---- ADD SOS FEATURES TO DATA ----
START: 2023-03-14 12:20:47.007334-07:00
FINISH: 2023-03-14 12:22:55.677526-07:00


,Season,TeamID,NumOT,TScore,TFGM,TFGA,TFGM3,TFGA3,TFTM,TFTA,...,OAstP,OStlP,OBlkP,OTOP,OffRtg,DefRtg,DayNum,OWP,OOWP,SOS
0,2010.0,3102.0,0.0,46.0,15.0,47.0,5.0,17.0,11.0,20.0,...,0.520000,0.147059,0.033333,0.263930,0.676471,0.953079,12.0,1.0,0.0,0.666667
1,2010.0,3103.0,0.0,63.0,23.0,54.0,5.0,9.0,12.0,19.0,...,0.550000,0.100575,0.133333,0.328571,0.905172,0.700000,12.0,0.0,1.0,0.333333
2,2010.0,3104.0,0.0,73.0,26.0,62.0,5.0,12.0,16.0,28.0,...,0.280000,0.051813,0.040000,0.250627,0.945596,0.852130,12.0,0.0,1.0,0.333333
3,2010.0,3105.0,0.0,61.0,22.0,57.0,3.0,12.0,14.0,21.0,...,0.633333,0.198939,0.133333,0.198939,0.809019,0.954907,12.0,1.0,0.0,0.666667
4,2010.0,3107.0,0.0,56.0,22.0,57.0,5.0,17.0,7.0,9.0,...,0.448276,0.190217,0.125000,0.261708,0.760870,1.046832,12.0,1.0,0.0,0.666667


In [91]:
season_end_data_l30 = continuous_data_l30.loc[continuous_data_l30.groupby(['Season','TeamID'])['DayNum'].idxmax()]
write_to_csv(season_end_data_l30, 'regular_season_end_l30.csv', 'Data/{}Transformed/'.format(bi))
season_end_data_l30.head()

,Season,TeamID,NumOT,TScore,TFGM,TFGA,TFGM3,TFGA3,TFTM,TFTA,...,OAstP,OStlP,OBlkP,OTOP,OffRtg,DefRtg,DayNum,OWP,OOWP,SOS
40846,2010.0,3102.0,0.0,50.571429,19.000000,53.285714,4.857143,18.142857,7.714286,10.428571,...,0.696517,0.158804,0.142276,0.223740,0.826716,1.229392,132.0,0.617837,0.461743,0.565805
40847,2010.0,3103.0,0.0,63.571429,23.714286,53.285714,4.285714,12.142857,11.857143,17.000000,...,0.559211,0.138949,0.100694,0.247418,0.966131,0.910069,132.0,0.578231,0.471817,0.542760
40848,2010.0,3104.0,0.0,60.333333,23.500000,57.166667,4.333333,13.166667,9.000000,13.333333,...,0.418919,0.167048,0.106061,0.220009,0.828375,0.956461,132.0,0.555556,0.432926,0.514679
40849,2010.0,3105.0,0.0,59.777778,19.222222,47.555556,3.222222,9.444444,18.111111,27.000000,...,0.552239,0.176879,0.093294,0.267912,0.849653,0.886405,132.0,0.492989,0.502376,0.496118
40850,2010.0,3106.0,0.0,57.100000,20.200000,52.100000,3.000000,9.800000,13.700000,21.000000,...,0.512048,0.130045,0.101655,0.275174,0.853513,0.830057,132.0,0.540635,0.484644,0.521971


## <u>Predict Winnners with Elo</u>

In [92]:
# Get basic results from all regular season games
season_data_compact = pd.read_csv('Data/{}RegularSeasonCompactResults.csv'.format(bi, bi))

In [93]:
# Merge Elo with compact results
season_elo_continuous = season_data_compact.merge(continuous_data_all[['Season','TeamID','TElo','TCarmElo','DayNum']], \
    left_on=['Season','WTeamID','DayNum'], right_on=['Season','TeamID','DayNum']).drop(columns=['TeamID'])
season_elo_continuous = season_elo_continuous.rename(columns={'TElo':'WElo','TCarmElo':'WCarmElo'})
season_elo_continuous = season_elo_continuous.merge(continuous_data_all[['Season','TeamID','TElo','TCarmElo','DayNum']], \
    left_on=['Season','LTeamID','DayNum'], right_on=['Season','TeamID','DayNum']).drop(columns=['TeamID'])
season_elo_continuous = season_elo_continuous.rename(columns={'TElo':'LElo','TCarmElo':'LCarmElo'})

# Look back at Elo from previous day (to introduce no training bias)
season_elo_continuous = pd.concat([season_elo_continuous[['Season','DayNum','WTeamID','LTeamID','WLoc']], \
           season_elo_continuous[['WElo','LElo','WCarmElo','LCarmElo']].shift(1)], axis=1).dropna()

# Add 100 to CarmElo of home team
season_elo_continuous['WCarmElo'] = season_elo_continuous.apply(lambda row: \
                                100 + row['WCarmElo'] if row['WLoc'] == 'H'else row['WCarmElo'], axis=1)
season_elo_continuous['LCarmElo'] = season_elo_continuous.apply(lambda row: \
                                100 + row['LCarmElo'] if row['WLoc'] == 'A' else row['LCarmElo'], axis=1)

# Calculate probability of WTeamID winning based on difference between Elos
season_elo_continuous['PElo'] = season_elo_continuous.apply(lambda row: \
                                            1 / (1 + 10**((row['LElo']-row['WElo'])/400)), axis=1)
season_elo_continuous['PCarmElo'] = season_elo_continuous.apply(lambda row: \
                                            1 / (1 + 10**((row['LCarmElo']-row['WCarmElo'])/400)), axis=1)

# Calculate success rate (accuracy, logloss) of each version of Elo
print('---- PREDICTING REGULAR SEASON OUTCOMES ----')
print('ELO ACCURACY:\t\t', len(season_elo_continuous[season_elo_continuous['PElo']>0.5])/len(season_elo_continuous))
print('CARMELO ACCURACY:\t', len(season_elo_continuous[season_elo_continuous['PCarmElo']>0.5])/len(season_elo_continuous))
print('ELO LOGLOSS:\t\t', -np.mean([math.log(x) for x in season_elo_continuous['PElo'].values]))
print('CARMELO LOGLOSS:\t', -np.mean([math.log(x) for x in season_elo_continuous['PCarmElo'].values]))

season_elo_continuous[['Season','DayNum','WTeamID','LTeamID','PElo','PCarmElo']].head()

---- PREDICTING REGULAR SEASON OUTCOMES ----
ELO ACCURACY:		 0.7125776648404107
CARMELO ACCURACY:	 0.7335673682818995
ELO LOGLOSS:		 0.5760664210108771
CARMELO LOGLOSS:	 0.5300060591670418


,Season,DayNum,WTeamID,LTeamID,PElo,PCarmElo
1,2010,12,3219,3464,0.5,0.639006
2,2010,12,3298,3189,0.5,0.369527
3,2010,12,3394,3104,0.5,0.622935
4,2010,12,3399,3102,0.5,0.514956
5,2010,12,3436,3301,0.5,0.367724


In [94]:
# Get basic results from all tournament games
tourney_data_compact = pd.read_csv('Data/{}NCAATourneyCompactResults.csv'.format(bi, bi))

# Merge Elo with compact results
tourney_elo_continuous = tourney_data_compact.merge(season_end_data_all[['Season','TeamID','TElo','TCarmElo']], \
    left_on=['Season','WTeamID'], right_on=['Season','TeamID']).drop(columns=['TeamID'])
tourney_elo_continuous = tourney_elo_continuous.rename(columns={'TElo':'WElo','TCarmElo':'WCarmElo'})
tourney_elo_continuous = tourney_elo_continuous.merge(season_end_data_all[['Season','TeamID','TElo','TCarmElo']], \
    left_on=['Season','LTeamID'], right_on=['Season','TeamID']).drop(columns=['TeamID'])
tourney_elo_continuous = tourney_elo_continuous.rename(columns={'TElo':'LElo','TCarmElo':'LCarmElo'})

# Calculate probability of WTeamID winning based on difference between Elos
tourney_elo_continuous['PElo'] = tourney_elo_continuous.apply(lambda row: \
                                            1 / (1 + 10**((row['LElo']-row['WElo'])/400)), axis=1)
tourney_elo_continuous['PCarmElo'] = tourney_elo_continuous.apply(lambda row: \
                                            1 / (1 + 10**((row['LCarmElo']-row['WCarmElo'])/400)), axis=1)

# Calculate success rate (accuracy, logloss) of each version of Elo
print('---- PREDICTING TOURNAMENT OUTCOMES ----')
print('ELO ACCURACY:\t\t', len(tourney_elo_continuous[tourney_elo_continuous['PElo']>0.5])/len(tourney_elo_continuous))
print('CARMELO ACCURACY:\t', len(tourney_elo_continuous[tourney_elo_continuous['PCarmElo']>0.5])/len(tourney_elo_continuous))
print('ELO LOGLOSS:\t\t', -np.mean([math.log(x) for x in tourney_elo_continuous['PElo'].values]))
print('CARMELO LOGLOSS:\t', -np.mean([math.log(x) for x in tourney_elo_continuous['PCarmElo'].values]))

tourney_elo_continuous[['Season','DayNum','WTeamID','LTeamID','PElo','PCarmElo']]

---- PREDICTING TOURNAMENT OUTCOMES ----
ELO ACCURACY:		 0.775
CARMELO ACCURACY:	 0.7710526315789473
ELO LOGLOSS:		 0.47095847610188624
CARMELO LOGLOSS:	 0.49633409495665104


,Season,DayNum,WTeamID,LTeamID,PElo,PCarmElo
0,2010,138,3124,3201,0.599769,0.505052
1,2010,140,3124,3207,0.457126,0.559743
2,2010,145,3124,3397,0.213716,0.355928
3,2010,147,3124,3181,0.273414,0.387037
4,2010,138,3173,3395,0.546807,0.493957
...,...,...,...,...,...,...
755,2022,138,3343,3246,0.474924,0.533492
756,2022,138,3397,3138,0.725621,0.569593
757,2022,140,3397,3125,0.836629,0.579332
758,2022,138,3416,3196,0.642656,0.702337


## <u>XGBoost Regression Model</u>

In [95]:
def rmse(y_obs, y_act):
    return np.sqrt(sum((y_act-y_obs)**2)/len(y_act))

def logloss(y_obs, y_act):
    return -np.mean([y*math.log(x) + (1-y)*math.log(1-x) for x,y in list(zip(y_obs, y_act)) if x < 1 and x > 0])

In [96]:
params = {'Outcome': {'max_depth': 3, 'min_child_weight': 1, 'colsample_bytree': 1.0, \
                     'subsample': 0.9, 'gamma': 0.7, 'reg_alpha': 100.0}, \
          'Score': {'max_depth': 3, 'min_child_weight': 3, 'colsample_bytree': 1.0, \
                     'subsample': 0.8, 'gamma': 0.1, 'reg_alpha': 100.0}}

In [97]:
cols = continuous_data_l30.columns[2:-4].append(continuous_data_l30.columns[-3:])

continuous_data_ext = continuous_data_all.merge(continuous_data_l30.rename(columns=\
                    dict(zip(cols, ['L'+x for x in cols]))), on=['Season','TeamID','DayNum'])
continuous_data_ext['DayNum'] = continuous_data_ext['DayNum'] + 1

matchup_data = season_data_compact[season_data_compact['Season']>=2010].drop(columns=['NumOT'])
matchup_data_W = matchup_data.merge(continuous_data_ext, left_on=['Season','WTeamID','DayNum'], \
                   right_on=['Season','TeamID','DayNum']).drop(columns=['TeamID'])
matchup_data_L = matchup_data.merge(continuous_data_ext, left_on=['Season','LTeamID','DayNum'], \
                   right_on=['Season','TeamID','DayNum']).drop(columns=['TeamID'])

cols = ['Season','DayNum','WTeamID','WScore','LTeamID','LScore','WLoc']
matchup_data1 = matchup_data_W.merge(matchup_data_L, on=cols).rename(columns=\
                                    {'WTeamID':'TeamID1','LTeamID':'TeamID2','WScore':'Score1','LScore':'Score2'})
matchup_data2 = matchup_data_L.merge(matchup_data_W, on=cols).rename(columns=\
                                    {'LTeamID':'TeamID1','WTeamID':'TeamID2','LScore':'Score1','WScore':'Score2'})
matchup_data = matchup_data1.append(matchup_data2[matchup_data1.columns]).reset_index(drop=True)

matchup_data['Win_Prob'] = matchup_data.apply(lambda row: 1/(1 + 10**((row['TCarmElo_y']-row['TCarmElo_x'])/400)), \
                                              axis=1)

matchup_data.head()

,Season,DayNum,TeamID1,Score1,TeamID2,Score2,WLoc,NumOT_x,TScore_x,TFGM_x,...,LOAstP_y,LOStlP_y,LOBlkP_y,LOTOP_y,LOffRtg_y,LDefRtg_y,LOWP_y,LOOWP_y,LSOS_y,Win_Prob
0,2010,13,3133,79,3227,64,H,0.0,73.0,28.0,...,0.666667,0.112045,0.035088,0.276074,0.686275,0.935583,1.0,0.0,0.666667,0.499409
1,2010,13,3142,90,3308,86,H,0.0,61.0,25.0,...,0.800000,0.114213,0.066667,0.288221,1.002538,0.902256,0.0,1.0,0.333333,0.475714
2,2010,13,3166,77,3172,58,H,0.0,77.0,27.0,...,0.500000,0.108959,0.117647,0.301932,0.750605,1.038647,1.0,0.0,0.666667,0.548531
3,2010,13,3177,77,3295,65,H,0.0,86.0,33.0,...,0.541667,0.123839,0.095238,0.258359,0.990712,0.942249,0.0,1.0,0.333333,0.509459
4,2010,13,3182,100,3310,69,A,0.0,74.0,28.0,...,0.615385,0.171233,0.055556,0.328638,0.901826,1.021127,1.0,0.0,0.666667,0.520702


In [98]:
r = 'Score'

cols = matchup_data.drop(columns=['TeamID1','TeamID2','Score1','Score2','WLoc']).columns
x_mat = np.matrix(matchup_data[cols])
y_mat = np.matrix(matchup_data['Score1']).T

x_train, x_test, y_train, y_test = train_test_split(x_mat, y_mat, test_size=0.3, random_state=4747)

model = xgb.XGBRegressor(learning_rate=0.01, n_estimators=10000, \
                max_depth=params[r]['max_depth'], \
                min_child_weight=params[r]['min_child_weight'], \
                subsample=params[r]['subsample'], \
                colsample_bytree=params[r]['colsample_bytree'],\
                gamma=params[r]['gamma'],\
                reg_alpha=params[r]['reg_alpha'])

model.fit(x_train, y_train, eval_set=[(x_test, y_test)], early_stopping_rounds=250, \
          eval_metric='rmse', verbose=False)

save_model_to_dir(model, 'xgbr_{}_{}_all.txt'.format(bracket_type, r.lower()), 'Models')

y_pred = model.predict(x_test)

score = rmse(np.array([x[0,0] for x in y_test]), y_pred)
print("RMSE: {}".format(score))

fi = pd.DataFrame(cols)
fi = fi.rename(columns={0:'col'}).join(pd.DataFrame(model.feature_importances_).rename(columns={0:'imp'}))

fi['imp'] = fi[['imp']].sort_values(by='imp', ascending=False).cumsum()

write_to_csv(fi, 'xgbr{}_{}_feature_importance.csv'.format(bracket_type, r.lower()), 'Features', index=None)

print("TOP FEATURES: {}\n".format(list(fi.sort_values(by='imp').col)[:5]))

RMSE: 10.373405580662885
TOP FEATURES: ['TScore_x', 'Win_Prob', 'OScore_y', 'LTScore_x', 'LOScore_y']



In [99]:
r = 'Score'

tourney_data_compact = pd.read_csv('Data/{}NCAATourneyCompactResults.csv'.format(bi, bi))

cols = season_end_data_l30.columns[2:-4].append(season_end_data_l30.columns[-3:])

season_end_data_ext = season_end_data_all.merge(season_end_data_l30.rename(columns=\
                    dict(zip(cols, ['L'+x for x in cols]))), on=['Season','TeamID','DayNum'])

tourney_xgbr_continuous = tourney_data_compact.merge(season_end_data_ext.drop(columns=['DayNum']), \
    left_on=['Season','WTeamID'], right_on=['Season','TeamID']).drop(columns=['TeamID'])
tourney_xgbr_continuous = tourney_xgbr_continuous.merge(season_end_data_ext.drop(columns=['DayNum']), \
    left_on=['Season','LTeamID'], right_on=['Season','TeamID']).drop(columns=['TeamID'])

tourney_xgbr_continuous = tourney_xgbr_continuous.rename(columns=\
                                {'WTeamID':'TeamID1','WScore':'Score1','LTeamID':'TeamID2','LScore':'Score2'})
tourney_xgbr_continuous['Win_Prob'] = tourney_xgbr_continuous.apply(lambda row: \
                                            1 / (1 + 10**((row['TCarmElo_y']-row['TCarmElo_x'])/400)), axis=1)
tourney_xgbr_continuous = tourney_xgbr_continuous[matchup_data.columns]

model = xgb.Booster()
model.load_model('Models/xgbr_{}_{}_all.txt'.format(bracket_type, r.lower()))

cols = tourney_xgbr_continuous.drop(columns=['TeamID1','TeamID2','Score1','Score2','WLoc']).columns

x_mat1 = xgb.DMatrix(tourney_xgbr_continuous[cols])
rpred1 = tourney_xgbr_continuous[['Season','TeamID1','TeamID2','Score1','Score2']]
rpred1 = rpred1.join(pd.DataFrame(model.predict(x_mat1))).rename(columns={0:'P{}1'.format(r)})

x_mat2 = xgb.DMatrix(tourney_xgbr_continuous.rename(columns=\
                                                dict([(x, x[:-1] + chr(1-(ord(x[-1])-ord('x')) + ord('x'))) \
                                                      for x in cols if x[-2:] == '_x' or x[-2:] == '_y']))[cols]\
                    .apply(lambda x: 1-x if x.name == 'Win_Prob' else x))
rpred2 = tourney_xgbr_continuous[['Season','TeamID1','TeamID2','Score1','Score2']]
rpred2 = rpred2.join(pd.DataFrame(model.predict(x_mat2))).rename(columns={0:'P{}2'.format(r)})

rpred = rpred1.merge(rpred2, on=['Season','TeamID1','TeamID2','Score1','Score2'])
    
season_data_std_score = season_data[['Season','WTeamID','WScore']]\
.rename(columns={'WTeamID':'TeamID','WScore':'Score'})\
.append(season_data[['Season','LTeamID','LScore']]\
.rename(columns={'LTeamID':'TeamID','LScore':'Score'}))\
.groupby(['Season','TeamID'], as_index=False).std()\
.rename(columns={'Score':'StdScore'})

rpred = rpred.merge(season_data_std_score, left_on=['Season','TeamID1'], right_on=['Season','TeamID'])
rpred = rpred.drop(columns=['TeamID']).rename(columns={'StdScore':'StdScore1'})
rpred = rpred.merge(season_data_std_score, left_on=['Season','TeamID2'], right_on=['Season','TeamID'])
rpred = rpred.drop(columns=['TeamID']).rename(columns={'StdScore':'StdScore2'})

rpred['Pred'] = rpred.apply(lambda row: norm.cdf(0, row['PScore2']-row['PScore1'], \
                                                 math.sqrt(row['StdScore1']*row['StdScore2'])), axis=1)
rpred['Outcome'] = rpred.apply(lambda row: 1 if row['Score1']>row['Score2'] else 0, axis=1)

print('---- PREDICTING TOURNAMENT OUTCOMES ----')
print('ACCURACY:\t', len(rpred[(rpred['Pred']>=0.5)])/len(rpred))
print('LOGLOSS:\t', logloss(rpred['Pred'].values, rpred['Outcome'].values))

rpred.head()

---- PREDICTING TOURNAMENT OUTCOMES ----
ACCURACY:	 0.7881578947368421
LOGLOSS:	 0.4419913161084762


,Season,TeamID1,TeamID2,Score1,Score2,PScore1,PScore2,StdScore1,StdScore2,Pred,Outcome
0,2010,3124,3201,69,55,66.800682,63.267342,15.942152,12.507725,0.598792,1
1,2010,3124,3207,49,33,61.962620,59.709736,15.942152,11.342576,0.566526,1
2,2010,3124,3397,77,62,56.500088,65.476631,15.942152,12.529602,0.262670,1
3,2010,3124,3181,51,48,57.483097,64.814346,15.942152,13.727438,0.310097,1
4,2010,3173,3395,67,66,67.464638,65.496689,10.041146,13.110012,0.568093,1


## XGBoost Classification Model

In [67]:
r = 'Outcome'

In [68]:
matchup_data_bin = matchup_data.copy()
matchup_data_bin['Win_Prob'] = matchup_data_bin.apply(lambda row: \
                                            1 / (1 + 10**((row['TCarmElo_y']-row['TCarmElo_x'])/400)), axis=1)
matchup_data_bin[r] = matchup_data_bin.apply(lambda row: 1 if row['Score1']>row['Score2'] else 0, axis=1)

print('-- {}: {} --'.format(r.upper(), datetime.now()))
x_mat = np.matrix(matchup_data_bin.drop(columns=['TeamID1','TeamID2','Score1','Score2','WLoc','Outcome']))
y_mat = np.matrix(matchup_data_bin[r]).T

x_train, x_test, y_train, y_test = train_test_split(x_mat, y_mat, test_size=0.3, random_state=4747)

model = xgb.XGBClassifier(objective='binary:logistic', learning_rate=0.01, n_estimators=10000, \
                max_depth=params['Score1']['max_depth'], \
                min_child_weight=params['Score1']['min_child_weight'], \
                subsample=params['Score1']['subsample'], \
                colsample_bytree=params['Score1']['colsample_bytree'],\
                gamma=params['Score1']['gamma'],\
                reg_alpha=params['Score1']['reg_alpha'])

model.fit(x_train, y_train, eval_set=[(x_test, y_test)], early_stopping_rounds=250, \
          eval_metric='logloss', verbose=False)

save_model_to_dir(model, 'xgbc_{}_{}_all.txt'.format(bracket_type, r.lower()), 'Models')

y_pred = model.predict_proba(x_test)

score = logloss([x[1] for x in y_pred], np.array([x[0,0] for x in y_test]))
print("LOGLOSS: {}".format(score))

fi = pd.DataFrame(matchup_data_bin.drop(columns=['TeamID1','TeamID2','Score1','Score2','WLoc','Outcome']).columns)
fi = fi.rename(columns={0:'col'}).join(pd.DataFrame(model.feature_importances_).rename(columns={0:'imp'}))

fi['imp'] = fi[['imp']].sort_values(by='imp', ascending=False).cumsum()

write_to_csv(fi, 'xgbc{}_{}_feature_importance.csv'.format(bracket_type, r.lower()), 'Features', index=None)

print("TOP FEATURES: {}\n".format(list(fi.sort_values(by='imp').col)[:5]))

-- OUTCOME: 2023-03-14 07:14:44.177289 --


/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  
/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Noah.Keshner/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column

LOGLOSS: 0.5534913623102751
TOP FEATURES: ['Win_Prob', 'OffRtg_y', 'TElo_y', 'TElo_x', 'OffRtg_x']



In [69]:
tourney_xgbc_continuous = tourney_xgbr_continuous.copy()
tourney_xgbc_continuous['Win_Prob'] = tourney_xgbc_continuous.apply(lambda row: \
                                            1 / (1 + 10**((row['TCarmElo_y']-row['TCarmElo_x'])/400)), axis=1)
tourney_xgbc_continuous[r] = tourney_xgbc_continuous.apply(lambda row: 1 if row['Score1']>row['Score2'] else 0, axis=1)
x_mat = xgb.DMatrix(tourney_xgbc_continuous.drop(columns=['TeamID1','TeamID2','Score1','Score2','WLoc','Outcome']))
cpred = tourney_xgbc_continuous[['Season','TeamID1','TeamID2','Score1','Score2','Outcome']]

model = xgb.Booster()
model.load_model('Models/xgbc_{}_{}_all.txt'.format(bracket_type, r.lower()))
cpred = cpred.join(pd.DataFrame(model.predict(x_mat))).rename(columns={0:'Pred'})


print('---- PREDICTING TOURNAMENT OUTCOMES ----')
print('ACCURACY:\t', (len(cpred[(cpred['Pred']>=0.5)]))/len(cpred))
print('LOGLOSS:\t', logloss(cpred['Pred'].values, cpred['Outcome'].values))

cpred.head()

---- PREDICTING TOURNAMENT OUTCOMES ----
ACCURACY:	 0.685
LOGLOSS:	 0.5681675738191767


,Season,TeamID1,TeamID2,Score1,Score2,Outcome,Pred
0,2010,1115,1457,61,44,1,0.512617
1,2010,1124,1358,68,59,1,0.770434
2,2010,1124,1330,76,68,1,0.611953
3,2010,1124,1388,72,49,1,0.637876
4,2010,1139,1431,77,59,1,0.438478


In [70]:
cpred.sort_values(by='Pred')

,Season,TeamID1,TeamID2,Score1,Score2,Outcome,Pred
188,2012,1313,1281,86,84,1,0.071303
436,2016,1292,1277,90,81,1,0.086239
596,2018,1420,1438,74,54,1,0.095670
694,2021,1331,1326,75,72,1,0.116529
244,2013,1195,1207,78,68,1,0.118078
...,...,...,...,...,...,...,...
491,2017,1437,1291,76,56,1,0.942777
678,2021,1124,1216,79,55,1,0.944273
125,2011,1326,1427,75,46,1,0.949552
345,2015,1246,1214,79,56,1,0.952664


## <u>Predict Current Tournament Results</u>

In [100]:
# Regular season stats
season_data = pd.read_csv('Data/{}RegularSeasonDetailedResults.csv'.format(bi, bi))
season_data_latest = season_data[season_data['Season']==2023].reset_index(drop=True)

# Elo stats
elo_dict_latest = get_elo_dict(season_data, min_season=2010, max_season=2023, min_team=MIN_TEAM, max_team=MAX_TEAM)
carmelo_dict_latest = get_carmelo_dict(season_data, min_season=2010, max_season=2023, \
                                       min_team=MIN_TEAM, max_team=MAX_TEAM)

# Stats from whole season
continuous_data_all_latest = get_continuous_data(season_data_latest, min_season=2023, max_season=2023)
matchup_dict_all_latest = get_matchup_dict(season_data_latest, min_season=2023, max_season=2023, \
                                           min_team=MIN_TEAM, max_team=MAX_TEAM)
opp_matchup_dict_all_latest = get_opp_matchup_dict(matchup_dict_all_latest, min_season=2023, max_season=2023, \
                                                   min_team=MIN_TEAM, max_team=MAX_TEAM)
wp_dict_all_latest = get_win_pct_dict(continuous_data_all_latest)
continuous_data_all_latest = add_sos_var(continuous_data_all_latest, wp_dict_all_latest, \
                                  matchup_dict_all_latest, opp_matchup_dict_all_latest)
continuous_data_all_latest = add_elo_var(continuous_data_all_latest, elo_dict_latest, \
                                         matchup_dict_all_latest, elo_var_name='Elo')
continuous_data_all_latest = add_elo_var(continuous_data_all_latest, carmelo_dict_latest, \
                                         matchup_dict_all_latest, elo_var_name='CarmElo')
continuous_data_all_latest = continuous_data_all_latest.apply(pd.to_numeric, errors='coerce', axis=1)
continuous_data_all_latest = continuous_data_all_latest.reset_index(drop=True).drop(columns=['Wins'])
season_end_data_all_latest = continuous_data_all_latest.loc[continuous_data_all_latest\
                                                            .groupby(['Season','TeamID'])['DayNum'].idxmax()]

# Stats from last 30 days
continuous_data_l30_latest = get_continuous_data(season_data_latest, min_season=2023, max_season=2023, last_n=30)
matchup_dict_l30_latest = get_matchup_dict(season_data_latest, min_season=2023, max_season=2023, last_n=30, \
                                           min_team=MIN_TEAM, max_team=MAX_TEAM)
opp_matchup_dict_l30_latest = get_opp_matchup_dict(matchup_dict_l30_latest, min_season=2023, max_season=2023, \
                                                   min_team=MIN_TEAM, max_team=MAX_TEAM)
wp_dict_l30_latest = get_win_pct_dict(continuous_data_l30_latest)
continuous_data_l30_latest = add_sos_var(continuous_data_l30_latest, wp_dict_l30_latest, \
                                         matchup_dict_l30_latest, opp_matchup_dict_l30_latest)
continuous_data_l30_latest = continuous_data_l30_latest.apply(pd.to_numeric, errors='coerce', axis=1)
continuous_data_l30_latest = continuous_data_l30_latest.reset_index(drop=True).drop(columns=['Wins'])
season_end_data_l30_latest = continuous_data_l30_latest.loc[continuous_data_l30_latest\
                                                            .groupby(['Season','TeamID'])['DayNum'].idxmax()]

cols = season_end_data_l30_latest.columns[2:-4].append(season_end_data_l30_latest.columns[-3:])
season_end_data_ext_latest = season_end_data_all_latest.merge(season_end_data_l30_latest.rename(columns=\
                    dict(zip(cols, ['L'+x for x in cols]))), on=['Season','TeamID','DayNum'])

season_end_data_ext_latest.head()

---- CALCULATE SIMPLE ELO SCORE ----
START: 2023-03-14 12:53:50.276928-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 13:12:20.136615-07:00
---- CALCULATE 538 ELO SCORE ----
START: 2023-03-14 13:12:20.361845-07:00
2010 season completed.
2011 season completed.
2012 season completed.
2013 season completed.
2014 season completed.
2015 season completed.
2016 season completed.
2017 season completed.
2018 season completed.
2019 season completed.
2020 season completed.
2021 season completed.
2022 season completed.
2023 season completed.
FINISH: 2023-03-14 13:40:06.989106-07:00
---- PROCESS CONTINOUS SEASON DATA ----
START: 2023-03-14 13:40:07.165914-07:00


/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(se

/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(se

/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(se

/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(se

...

/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)

/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)

/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)&(season_data['DayNum']>=daynum-days_back)]
/var/folders/81/_gx_gpcn35711xr559qh2pnh0000gp/T/ipykernel_9096/3307344716.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = data[(data['Season']==seas)&(data['DayNum']<daynum)

2023 season completed.
FINISH: 2023-03-14 13:41:46.423364-07:00
---- COLLECT OPPONENT MATCHUP HISTORY ----
START: 2023-03-14 13:41:46.429173-07:00
2023 season completed.
FINISH: 2023-03-14 13:41:46.516111-07:00
---- ADD SOS FEATURES TO DATA ----
START: 2023-03-14 13:41:47.299389-07:00
FINISH: 2023-03-14 13:41:58.441614-07:00


,Season,TeamID,NumOT,TScore,TFGM,TFGA,TFGM3,TFGA3,TFTM,TFTA,...,LOORP,LOAstP,LOStlP,LOBlkP,LOTOP,LOffRtg,LDefRtg,LOWP,LOOWP,LSOS
0,2023.0,3101.0,0.037037,69.851852,25.555556,58.000000,8.296296,24.444444,10.444444,15.111111,...,0.265918,0.527273,0.115311,0.058632,0.212392,1.016976,0.958160,0.487654,0.485780,0.487030
1,2023.0,3102.0,0.000000,63.275862,23.413793,58.620690,5.310345,16.793103,11.137931,15.482759,...,0.258427,0.475410,0.095714,0.092715,0.229453,0.880150,1.005423,0.670068,0.439394,0.593177
2,2023.0,3103.0,0.035714,67.000000,23.357143,55.107143,6.250000,20.250000,14.035714,18.250000,...,0.288538,0.602564,0.122863,0.080139,0.153739,0.929487,1.058700,0.564732,0.477833,0.535766
3,2023.0,3104.0,0.000000,69.866667,25.066667,57.066667,8.900000,23.400000,10.833333,15.066667,...,0.293578,0.471698,0.107374,0.170616,0.230482,0.976499,0.930233,0.365646,0.542717,0.424670
4,2023.0,3105.0,0.068966,56.793103,19.758621,55.965517,4.310345,16.413793,12.965517,18.413793,...,0.240506,0.467949,0.094147,0.098639,0.185639,0.859599,0.805794,0.397959,0.563323,0.453081


In [101]:
r = 'Score'

tourney_predictions = pd.read_csv('Data/SampleSubmission2023.csv'.format(bi, bi))

tourney_predictions['Season'] = tourney_predictions.apply(lambda row: int(row['ID'][:4]), axis=1)
tourney_predictions['TeamID1'] = tourney_predictions.apply(lambda row: int(row['ID'][5:9]), axis=1)
tourney_predictions['TeamID2'] = tourney_predictions.apply(lambda row: int(row['ID'][10:14]), axis=1)

tourney_predictions = tourney_predictions.merge(season_end_data_ext_latest, \
                        left_on=['Season','TeamID1'], right_on=['Season','TeamID']).drop(columns=['TeamID','DayNum'])
tourney_predictions = tourney_predictions.merge(season_end_data_ext_latest, \
                        left_on=['Season','TeamID2'], right_on=['Season','TeamID']).drop(columns=['TeamID'])

tourney_predictions['Win_Prob'] = tourney_predictions.apply(lambda row: \
                                            1 / (1 + 10**((row['TCarmElo_y']-row['TCarmElo_x'])/400)), axis=1)

model = xgb.Booster()
model.load_model('Models/xgbr_{}_{}_all.txt'.format(bracket_type, r.lower()))

cols = tourney_xgbr_continuous.drop(columns=['TeamID1','TeamID2','Score1','Score2','WLoc']).columns

x_mat1 = xgb.DMatrix(tourney_predictions[cols])
tourney_predictions['P{}1'.format(r)] = model.predict(x_mat1)

x_mat2 = xgb.DMatrix(tourney_predictions.rename(columns=\
                                                dict([(x, x[:-1] + chr(1-(ord(x[-1])-ord('x')) + ord('x'))) \
                                                 for x in cols if x[-2:] == '_x' or x[-2:] == '_y']))[cols]\
                    .apply(lambda x: 1-x if x.name == 'Win_Prob' else x))
tourney_predictions['P{}2'.format(r)] = model.predict(x_mat2)

season_data_std_score = season_data_latest[['Season','WTeamID','WScore']]\
.rename(columns={'WTeamID':'TeamID','WScore':'Score'})\
.append(season_data_latest[['Season','LTeamID','LScore']]\
.rename(columns={'LTeamID':'TeamID','LScore':'Score'}))\
.groupby(['Season','TeamID'], as_index=False).std()\
.rename(columns={'Score':'StdScore'})

tourney_predictions = tourney_predictions.merge(season_data_std_score, \
                                                left_on=['Season','TeamID1'], right_on=['Season','TeamID'])
tourney_predictions = tourney_predictions.drop(columns=['TeamID']).rename(columns=\
                                                                                    {'StdScore':'StdScore1'})
tourney_predictions = tourney_predictions.merge(season_data_std_score, \
                                                left_on=['Season','TeamID2'], right_on=['Season','TeamID'])
tourney_predictions = tourney_predictions.drop(columns=['TeamID']).rename(columns=\
                                                                                    {'StdScore':'StdScore2'})

tourney_predictions['Pred'] = tourney_predictions.apply(lambda row: \
                norm.cdf(0, row['PScore2']-row['PScore1'], math.sqrt(row['StdScore1']*row['StdScore2'])), axis=1)

write_to_csv(tourney_predictions[['ID','Pred']], f'{bracket_type}_{r}_submssion.csv', 'Output')

tourney_predictions.head()

,ID,Pred,Season,TeamID1,TeamID2,NumOT_x,TScore_x,TFGM_x,TFGA_x,TFGM3_x,...,LOffRtg_y,LDefRtg_y,LOWP_y,LOOWP_y,LSOS_y,Win_Prob,PScore1,PScore2,StdScore1,StdScore2
0,2023_3101_3102,0.555828,2023,3101,3102,0.037037,69.851852,25.555556,58.00000,8.296296,...,0.880150,1.005423,0.670068,0.439394,0.593177,0.554322,67.649727,66.108459,11.674112,10.322986
1,2023_3101_3103,0.590157,2023,3101,3103,0.037037,69.851852,25.555556,58.00000,8.296296,...,0.929487,1.058700,0.564732,0.477833,0.535766,0.462243,71.399887,68.903023,11.674112,10.277628
2,2023_3102_3103,0.519936,2023,3102,3103,0.000000,63.275862,23.413793,58.62069,5.310345,...,0.929487,1.058700,0.564732,0.477833,0.535766,0.408670,66.705673,66.190742,10.322986,10.277628
3,2023_3101_3104,0.205166,2023,3101,3104,0.037037,69.851852,25.555556,58.00000,8.296296,...,0.976499,0.930233,0.365646,0.542717,0.424670,0.195924,61.824314,72.529976,11.674112,14.483602
4,2023_3102_3104,0.143302,2023,3102,3104,0.000000,63.275862,23.413793,58.62069,5.310345,...,0.976499,0.930233,0.365646,0.542717,0.424670,0.163814,56.969326,69.999046,10.322986,14.483602


In [102]:
cols = ['Season','TeamID1','TeamID2','Pred','Win_Prob','PScore1','PScore2','StdScore1','StdScore2']
write_to_csv(tourney_predictions[cols], \
             'predictions_2023.csv', 'Output/{}/Regression/'.format(bi+bracket_type[1:]))

In [107]:
mens_pred = pd.read_csv('Output/mens_score_submssion.csv')
womens_pred = pd.read_csv('Output/womens_score_submssion.csv')

submission_pred = pd.concat([mens_pred, womens_pred], axis=0)
submission_pred.to_csv('Output/Submission2023.csv', index=None)

submission_pred

,ID,Pred
0,2023_1101_1102,0.484804
1,2023_1101_1103,0.230862
2,2023_1102_1103,0.239152
3,2023_1101_1104,0.042546
4,2023_1102_1104,0.052077
...,...,...
64975,2023_3472_3477,0.578885
64976,2023_3473_3477,0.175990
64977,2023_3474_3477,0.218617
64978,2023_3475_3477,0.504017


In [106]:
!kaggle competitions submit -c march-machine-learning-mania-2023 -f Output/Submission2023.csv -m "Predictions using XGBoost Regressor."

zsh:1: command not found: kaggle
